In [43]:
from unittest.mock import inplace

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization
import matplotlib.pyplot as plt
import re
%matplotlib inline


data = pd.DataFrame( pd.read_excel('./breast-cancer.xls') )  # Data Import from local/ Drive file

data['inv-nodes'].to_string(index=False)

################# Data sanitisation #################
total_rows = len(data)
print("Data size before sanitisation, Total: ", total_rows)
data.dropna() # Drops all NA cells by row and saves it back to source data


print("Removed, Rows with NA: ", total_rows - len(data))
for x in data.index:

    if re.search( r"^[0-9]{2}-[0-9]{2}$",  str(data.loc[x, "age"]) ) is None: #Using RegEx to confirm age pattern is correct, removes row that dont conform
        data.drop(index=x, inplace=True)
    elif re.search(r"^(ge40)$|^(premeno)$|^(lt40)$", str(data.loc[x, "menopause"])) is None:  #Using RegEx to confirm menopause name is only 1 of three
        data.drop(index=x, inplace=True)
    elif re.search(r"^[0-9]{1,2}-[0-9]{1,2}$", str(data.loc[x, "tumor-size"]) ) is None:  #Using RegEx to confirm tumor size is formated correctly
        data.drop(index=x, inplace=True)
    elif re.search(r"^[0-9]{1,2}-[0-9]{1,2}$", str(data.loc[x, "inv-nodes"])) is None: #Using RegEx to confirm node-caps is only yes or no Upper or lower case
        data.drop(index=x, inplace=True)
    elif re.match(r'^([Yy][eE][Ss]|[Nn][oO])$', str(data.loc[x, "node-caps"])) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(index=x, inplace=True)
    elif re.search(r"^1$|^2$|^3$", str(data.loc[x, "deg-malig"])) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(index=x, inplace=True)
    elif re.search(r"^([Ll][eE][fF][tT]|[Rr][Ii][Gg][hH][tT])$", str(data.loc[x, "breast"])) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(index=x)
    elif re.search(r"^(left_up|left_low|central|right_up|right_low)$", str(data.loc[x, "breast-quad"]), re.IGNORECASE) is None:  #Using RegEx to confirm inv-nodes count is a number range
        data.drop(index=x)
data.reset_index(drop=True, inplace=True)  # Re indexes entire dataframe to fix deleted rows
print("Total removal rows: " , total_rows - len(data))
################# Data sanitisation #################

################# Data Formating #################


targets = data[['menopause']].replace(['premeno','ge40','lt40'],[0,1,2]) # convertion of menopause table to int values to be used as primary ancher for the nural net
print(targets.sample(n=5))

################# Data Formating #################

################# Training Data Masking #################
training_data_per =  90/100.0

data['train'] = np.random.rand(len(data)) < training_data_per # Creates random array for each row  of Data, true or false using the fraction
Training_Data = data[data.train == 1] # Appends train Data with the array for each row, effectively showing which row at random will be removed

Training_Data = Training_Data.drop('train', axis=1).sample(frac=1)
################# Data Test Masking #################

################# Testing Data Masking #################
Testing_Data = data[data.train == 0]

Testing_Data = Testing_Data.drop('train', axis=1)

X = train.to_numpy()    ### OR .values[:,:4]
y = np.array([targets[int(x)] for x in train.values[:,4:5]])

################# Testing Data Masking #################
################# Testing Data Masking #################

#data.sample(len(data))

Data size before sanitisation, Total:  286
Removed, Rows with NA:  0
Total removal rows:  99
     menopause
122          2
78           1
109          0
37           1
119          1


C:\Users\vladl\AppData\Local\Temp\ipykernel_7476\3349841021.py:47: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  targets = data[['menopause']].replace(['premeno','ge40','lt40'],[0,1,2]) # convertion of menopause table to int values to be used as primary ancher for the nural net
C:\Users\vladl\AppData\Local\Temp\ipykernel_7476\3349841021.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y = np.array([targets[int(x)] for x in train.values[:,4:5]])


ValueError: invalid literal for int() with base 10: 'no'